In [88]:
# переконвертировать генбанк в gff

from BCBio import GFF
from Bio import SeqIO

gbf_file = "/mnt/projects/sochkalova/Psittacus/phyl/other_parrots.gbf"
gff_file = "/mnt/projects/sochkalova/Psittacus/phyl/other_parrots.gff"
gbf_handle = open(gbf_file)
gff_handle = open(gff_file, "w")

GFF.write(SeqIO.parse(gbf_handle, "genbank"), gff_handle)

gbf_handle.close()
gff_handle.close()


In [151]:
# из gff непситтакусов сделать удобный для пользования файл с координатами генов

ncbi_id = ''
id_dict = {}
counter = 0
cds_amount = 0
rna_amount = 0
gff_file = "/mnt/projects/sochkalova/Psittacus/phyl/other_parrots.gff"
out_file = "/mnt/projects/sochkalova/Psittacus/phyl/cds_rna_other_parrots.tsv"
with open(gff_file) as infile:
    with open(out_file, 'w') as outfile:
        for line in infile:
            line = line.strip().split('\t')
            if '##sequence-region' in line[0]:
                if ncbi_id:
                    header = '\n>%s %s\n' % (ncbi_id, species)
                    print(header, end=' ')
                    print(cds_amount, rna_amount)
                    outfile.write(header)
                    counter += 1
                    ncbi_id = line[0].split()[1]
                    cds_amount = 0
                    rna_amount = 0
                    for gene, coordinates in id_dict.items():
                        s = '%s\t%s\t%s\t%s\n' % (coordinates.split()[0], coordinates.split()[1], gene.split()[0], gene.split()[1])
                        outfile.write(s)
                    id_dict = {}
                    continue
                    
                if not ncbi_id:
                    ncbi_id = line[0].split()[1]
                    continue
                    
            if line[0] == ncbi_id:
                if line[1] == 'annotation':
                    if ncbi_id.startswith('NC') or ncbi_id.startswith('MG'):
                        species = line[8].split(';')[6].replace('organism=','')
                        continue
                    else:
                        species = line[8].split(';')[5].replace('organism=','')
                        continue
                    
                if line[2] == 'rRNA':
                    gene_name = 'rRNA {}'.format(line[8][-17:].replace(' ','_'))
                    if gene_name not in id_dict.keys():
                        id_dict[gene_name] = "%s %s" % (line[3], line[4])
                        rna_amount += 1
                        continue
                        
                if line[2] == 'CDS':
                    if ncbi_id.startswith('NC'):
                        gene_name = 'CDS {}'.format(line[8].split(';')[2].replace('gene=',''))
                        if gene_name not in id_dict.keys():
                            id_dict[gene_name] = "%s %s" % (line[3], line[4])
                            cds_amount += 1
                            continue
                    else:
                        gene_name = 'CDS {}'.format(line[8].split(';')[1].replace('gene=',''))
                        if gene_name not in id_dict.keys():
                            id_dict[gene_name] = "%s %s" % (line[3], line[4])
                            cds_amount += 1
                            continue
            
        if ncbi_id:
            header = '\n>%s %s\n' % (ncbi_id, species)
            print(header, end=' ')
            print(cds_amount, rna_amount)
            outfile.write(header)
            counter += 1
            for gene, coordinates in id_dict.items():
                s = '%s\t%s\t%s\t%s\n' % (coordinates.split()[0], coordinates.split()[1], gene.split()[0], gene.split()[1])
                outfile.write(s)
                        
#print(counter)


>NC_045369.1 Agapornis lilianae
 13 2

>NC_011708.1 Agapornis roseicollis
 13 2

>NC_034679.1 Amazona ventralis
 13 2

>NC_029319.1 Ara ararauna
 13 2

>NC_045076.1 Ara macao
 13 2

>JX215256.1 Aratinga mitrata mitrata
 13 2

>NC_045371.1 Aratinga nenday
 13 2

>NC_026039.1 Aratinga solstitialis
 13 2

>NC_027841.1 Coracopsis vasa
 13 2

>NC_015197.1 Eupsittula pertinax chrysogenys
 13 2

>NC_026031.1 Guaruba guarouba
 13 2

>NC_040142.1 Cacatua pastinator
 13 2

>MF977813.1 Poicephalus gulielmi gulielmi
 13 2

>NC_044083.1 Poicephalus senegalus
 13 2

>NC_020325.1 Psittacara acuticaudatus acuticaudatus
 13 2

>NC_021764.1 Psittacara brevipes
 13 2


In [91]:
# распарсить мультифасты с непситтакусами и пситтакусами и сложить их в один словарь

fasta = {} 
fasta_file = '/mnt/projects/sochkalova/Psittacus/phyl/other_parrots.mfa' 
header = None                             
with open(fasta_file) as fh:              
    for i, line in enumerate(fh):         
        line = line.strip()               
        if line.startswith(">"):          
            if header:                    
                fasta[header] = "".join(seq) 
            header = line             
            seq = []                      
        else:
            seq.append(line)              
    if header:                            
        fasta[header] = "".join(seq)   


fasta_file = '/mnt/projects/sochkalova/Psittacus/phyl/Psittacus_all.mfa'          
header = None                             
with open(fasta_file) as fh:              
    for i, line in enumerate(fh):         
        line = line.strip()               
        if line.startswith(">"):          
            if header:                    
                fasta[header] = "".join(seq) 
            header = line             
            seq = []                      
        else:
            seq.append(line)              
    if header:                            
        fasta[header] = "".join(seq)  
        
print(*fasta, sep='\n')

>NC_045369.1 Agapornis lilianae
>NC_011708.1 Agapornis roseicollis
>NC_034679.1 Amazona ventralis
>NC_029319.1 Ara ararauna
>NC_045076.1 Ara macao
>JX215256.1 Aratinga mitrata mitrata
>NC_045371.1 Aratinga nenday
>NC_026039.1 Aratinga solstitialis
>NC_027841.1 Coracopsis vasa
>NC_015197.1 Eupsittula pertinax chrysogenys
>NC_026031.1 Guaruba guarouba
>NC_040142.1 Cacatua pastinator
>MF977813.1 Poicephalus gulielmi gulielmi
>NC_044083.1 Poicephalus senegalus
>NC_020325.1 Psittacara acuticaudatus acuticaudatus
>NC_021764.1 Psittacara brevipes
>Psittacus2
>Psittacus1
>Psittacus5
>Psittacus10
>Psittacus11_hyb
>Psittacus9
>Psittacus8
>Psittacus12
>Psittacus15


In [92]:
# дозаписать в файл с координатами генов координаты для пситтакусов
ncbi_id = ''
id_dict = {}
counter = 0
cds_amount = 0
rna_amount = 0
gff_file = "/mnt/projects/sochkalova/Psittacus/phyl/Psittacus_all.gff"
out_file = "/mnt/projects/sochkalova/Psittacus/phyl/cds_rna_other_parrots.tsv"
with open(gff_file) as infile:
    with open(out_file, 'a') as outfile:
        for line in infile:
            line = line.strip().split('\t')
            
            if line[0].startswith('>') and ncbi_id:
                header = '\n>%s\n' % (ncbi_id)
                print(header)
                print(cds_amount, rna_amount)
                outfile.write(header)
                counter += 1
                ncbi_id = line[0][1:]
                cds_amount = 0
                rna_amount = 0
                for gene, coordinates in id_dict.items():
                    s = '%s\t%s\t%s\t%s\n' % (coordinates.split()[0], coordinates.split()[1], gene.split()[0], gene.split()[1])
                    outfile.write(s)
                id_dict = {}
                continue
                    
            if line[0].startswith('>') and not ncbi_id:
                ncbi_id = line[0][1:]
                continue
                    
            if line[0] == ncbi_id:                  
                if line[2] == 'rRNA':
                    gene_name = 'rRNA {}'.format(line[8].replace('Name=rrnS','12S_ribosomal_RNA').replace('Name=rrnL','16S_ribosomal_RNA'))
                    if gene_name not in id_dict.keys():
                        id_dict[gene_name] = "%s %s" % (line[3], line[4])
                        rna_amount += 1
                        continue
                        
                if line[2] == 'gene':
                    gene_name = 'CDS {}'.format(line[8].replace('Name=',''))
                    if gene_name not in id_dict.keys():
                        id_dict[gene_name] = "%s %s" % (line[3], line[4])
                        cds_amount += 1
                        continue
                        
        if ncbi_id:
            header = '\n>%s\n' % (ncbi_id)
            print(header)
            print(cds_amount, rna_amount)
            outfile.write(header)
            counter += 1
            for gene, coordinates in id_dict.items():
                s = '%s\t%s\t%s\t%s\n' % (coordinates.split()[0], coordinates.split()[1], gene.split()[0], gene.split()[1])
                outfile.write(s)
                        
print(counter)


>Psittacus10

13 2

>Psittacus11_hyb

13 2

>Psittacus12

13 2

>Psittacus15

13 2

>Psittacus1

13 2

>Psittacus2

13 2

>Psittacus5

13 2

>Psittacus8

13 2

>Psittacus9

13 2
9


In [112]:
# для каждого гена в списке создать отдельный файл где будет последовательность нуклеотидов этого гена каждого попугая

coordinates_file = "/mnt/projects/sochkalova/Psittacus/phyl/cds_rna_other_parrots.tsv"
genes_list = ['12S_ribosomal_RNA', '16S_ribosomal_RNA', 'ATP6', 'ATP8', 'COX1', 'COX2', 'COX3', 'CYTB','ND1', 'ND2', 'ND3', 'ND4', 'ND4L', 'ND5', 'ND6']
for gene in genes_list:
    #print('\n'+gene)
    _id = ''
    counter = 0
    gene_output_file = "/mnt/projects/sochkalova/Psittacus/phyl/genes/%s.mfa" % gene
    print(gene_output_file)
    with open(gene_output_file, 'w') as output:
         with open(coordinates_file) as fh:
            for line in fh:
                line = line.strip()
                if line.startswith('>'):
                    _id = line
                    continue
                if len(line.split('\t')) > 3:
                    if line.split('\t')[3] == gene:
                        start = int(line.split('\t')[0])
                        end = int(line.split('\t')[1])
                        seq_frag = fasta[_id][start:end+1]
                        header = '\n>%s\n' % (_id[1:]) 
                        output.write(header)
                        counter += 1
                        print(counter, _id, sep=' ')
                        output.write(seq_frag)
                    else:
                        continue

/mnt/projects/sochkalova/Psittacus/phyl/genes/12S_ribosomal_RNA.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/16S_ribosomal_RNA.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ATP6.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ATP8.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/COX1.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/COX2.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/COX3.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/CYTB.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ND1.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ND2.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ND3.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ND4.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ND4L.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ND5.mfa
/mnt/projects/sochkalova/Psittacus/phyl/genes/ND6.mfa


In [ ]:
#script to remove fucking spaces from gblocks output

for gene in genes_list:
    gblocks_output = "/mnt/projects/sochkalova/Psittacus/phyl/genes/%s_aligned.mfa-gb" % gene
    no_gaps_file = "/mnt/projects/sochkalova/Psittacus/phyl/genes/%s_aligned_conserv.mfa" % gene
    with open(gblocks_output) as ff:
        with open(no_gaps_file, 'w') as fh:
            for line in ff:
                line = line.strip()
                if line.startswith('>'):
                    fh.write('\n'+line+'\n')
                else:
                    line = line.split()
                    for block in line:
                        fh.write(block)

In [148]:
# concatenation script

final_file = '/mnt/projects/sochkalova/Psittacus/phyl/rrn_cds_aligned_concatenated.mfa' 
with open(final_file,  'w') as file:
    for bird_id in fasta:
        header = '\n\n%s\n' % (bird_id.replace(' ', '_'))
        file.write(header)
        print()
        print(bird_id)
        gene_len = 0
        for gene in genes_list:
            header = None
            gene_file = "/mnt/projects/sochkalova/Psittacus/phyl/genes/%s_aligned_conserv.mfa" % gene
            with open(gene_file) as fh:
                for line in fh:
                    line = line.strip()
                    if line == bird_id:
                        header = True
                        #print(line)
                        continue
                    if header:
                        if line.startswith('>'):
                            print(gene_len, end = ' ')
                            break
                        print(gene, end = ' ')
                        file.write(line)
                        gene_len += len(line)
                        


>NC_045369.1 Agapornis lilianae
12S_ribosomal_RNA 954 16S_ribosomal_RNA 2487 ATP6 3168 ATP8 3328 COX1 4867 COX2 5542 COX3 6325 CYTB 7459 ND1 8398 ND2 9424 ND3 9773 ND4 11156 ND4L 11450 ND5 13257 ND6 13770 
>NC_011708.1 Agapornis roseicollis
12S_ribosomal_RNA 954 16S_ribosomal_RNA 2487 ATP6 3168 ATP8 3328 COX1 4867 COX2 5542 COX3 6325 CYTB 7459 ND1 8398 ND2 9424 ND3 9773 ND4 11156 ND4L 11450 ND5 13257 ND6 13770 
>NC_034679.1 Amazona ventralis
12S_ribosomal_RNA 954 16S_ribosomal_RNA 2487 ATP6 3168 ATP8 3328 COX1 4867 COX2 5542 COX3 6325 CYTB 7459 ND1 8398 ND2 9424 ND3 9773 ND4 11156 ND4L 11450 ND5 13257 ND6 13770 
>NC_029319.1 Ara ararauna
12S_ribosomal_RNA 954 16S_ribosomal_RNA 2487 ATP6 3168 ATP8 3328 COX1 4867 COX2 5542 COX3 6325 CYTB 7459 ND1 8398 ND2 9424 ND3 9773 ND4 11156 ND4L 11450 ND5 13257 ND6 13770 
>NC_045076.1 Ara macao
12S_ribosomal_RNA 954 16S_ribosomal_RNA 2487 ATP6 3168 ATP8 3328 COX1 4867 COX2 5542 COX3 6325 CYTB 7459 ND1 8398 ND2 9424 ND3 9773 ND4 11156 ND4L 11450 ND5

In [147]:
string = '0 12S_ribosomal_RNA 954 16S_ribosomal_RNA 2487 ATP6 3168 ATP8 3328 COX1 4867 COX2 5542 COX3 6325 CYTB 7459 ND1 8398 ND2 9424 ND3 9773 ND4 11156 ND4L 11450 ND5 13257 ND6 13770 '
string = string.split()
for i in range(len(string)):
    if i != 0 and string[i].isdigit():
        start = int(string[i-2])+1
        end = int(string[i])
        line = '%s = %d-%d;' % (string[i-1], start, end)
        print(line)

12S_ribosomal_RNA = 1-954;
16S_ribosomal_RNA = 955-2487;
ATP6 = 2488-3168;
ATP8 = 3169-3328;
COX1 = 3329-4867;
COX2 = 4868-5542;
COX3 = 5543-6325;
CYTB = 6326-7459;
ND1 = 7460-8398;
ND2 = 8399-9424;
ND3 = 9425-9773;
ND4 = 9774-11156;
ND4L = 11157-11450;
ND5 = 11451-13257;
ND6 = 13258-13770;
